# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 7:40AM,240799,19,8212472,"Innogenix, LLC",Released
1,Jun 30 2022 7:34AM,240798,19,8212473,"Innogenix, LLC",Released
2,Jun 30 2022 7:34AM,240798,19,8212474,"Innogenix, LLC",Released
3,Jun 30 2022 7:34AM,240798,19,8212475,"Innogenix, LLC",Released
4,Jun 30 2022 7:34AM,240798,19,8212476,"Innogenix, LLC",Released
5,Jun 30 2022 7:34AM,240798,19,8212477,"Innogenix, LLC",Released
6,Jun 30 2022 7:34AM,240798,19,8212478,"Innogenix, LLC",Released
7,Jun 30 2022 7:34AM,240798,19,8212479,"Innogenix, LLC",Released
8,Jun 30 2022 7:34AM,240798,19,8212480,"Innogenix, LLC",Released
9,Jun 30 2022 7:34AM,240798,19,8212481,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,240793,Released,1
23,240795,Released,1
24,240797,Executing,1
25,240798,Released,15
26,240799,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240793,NaN,NaN,1.0
240795,NaN,NaN,1.0
240797,NaN,1.0,NaN
240798,NaN,NaN,15.0
240799,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240695,0.0,0.0,1.0
240743,0.0,1.0,0.0
240745,1.0,7.0,13.0
240750,0.0,0.0,26.0
240751,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240695,0,0,1
240743,0,1,0
240745,1,7,13
240750,0,0,26
240751,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240695,0,0,1
1,240743,0,1,0
2,240745,1,7,13
3,240750,0,0,26
4,240751,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240695,,,1
1,240743,,1,
2,240745,1,7,13
3,240750,,,26
4,240751,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC"
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC"
16,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc."
17,Jun 29 2022 4:37PM,240795,16,ACG North America LLC
18,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC"
19,Jun 29 2022 4:12PM,240792,19,"NAPP Technologies, LLC"
20,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc."
21,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc."
22,Jun 29 2022 3:30PM,240784,21,"NBTY Global, Inc."
23,Jun 29 2022 3:26PM,240781,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC",,,1
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",,,15
2,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc.",,1,
3,Jun 29 2022 4:37PM,240795,16,ACG North America LLC,,,1
4,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC",,,1
5,Jun 29 2022 4:12PM,240792,19,"NAPP Technologies, LLC",,,1
6,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc.",,,1
7,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc.",,,1
8,Jun 29 2022 3:30PM,240784,21,"NBTY Global, Inc.",,,1
9,Jun 29 2022 3:26PM,240781,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC",1,,
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",15,,
2,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc.",,1,
3,Jun 29 2022 4:37PM,240795,16,ACG North America LLC,1,,
4,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC",1,,
5,Jun 29 2022 4:12PM,240792,19,"NAPP Technologies, LLC",1,,
6,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc.",1,,
7,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc.",1,,
8,Jun 29 2022 3:30PM,240784,21,"NBTY Global, Inc.",1,,
9,Jun 29 2022 3:26PM,240781,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC",1,,
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",15,,
2,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc.",,1,
3,Jun 29 2022 4:37PM,240795,16,ACG North America LLC,1,,
4,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC",1.0,NaN,NaN
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",15.0,NaN,NaN
2,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc.",NaN,1.0,NaN
3,Jun 29 2022 4:37PM,240795,16,ACG North America LLC,1.0,NaN,NaN
4,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 7:40AM,240799,19,"Innogenix, LLC",1.0,0.0,0.0
1,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",15.0,0.0,0.0
2,Jun 30 2022 7:29AM,240797,22,"NBTY Global, Inc.",0.0,1.0,0.0
3,Jun 29 2022 4:37PM,240795,16,ACG North America LLC,1.0,0.0,0.0
4,Jun 29 2022 4:17PM,240793,19,"NAPP Technologies, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,481517,2.0,0.0,0.0
16,240795,1.0,0.0,0.0
17,240768,1.0,0.0,0.0
19,1444646,23.0,0.0,0.0
20,1444555,72.0,10.0,4.0
21,1203910,4.0,1.0,0.0
22,481540,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,481517,2.0,0.0,0.0
1,16,240795,1.0,0.0,0.0
2,17,240768,1.0,0.0,0.0
3,19,1444646,23.0,0.0,0.0
4,20,1444555,72.0,10.0,4.0
5,21,1203910,4.0,1.0,0.0
6,22,481540,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,2.0,0.0,0.0
1,16,1.0,0.0,0.0
2,17,1.0,0.0,0.0
3,19,23.0,0.0,0.0
4,20,72.0,10.0,4.0
5,21,4.0,1.0,0.0
6,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,2.0
1,16,Released,1.0
2,17,Released,1.0
3,19,Released,23.0
4,20,Released,72.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,10.0,1.0,2.0
Released,2.0,1.0,1.0,23.0,72.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,10.0,1.0,2.0
2,Released,2.0,1.0,1.0,23.0,72.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,10.0,1.0,2.0
2,Released,2.0,1.0,1.0,23.0,72.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()